In [1]:
import keras
import load
from keras.preprocessing.sequence import pad_sequences
from load import sentence2vector
from load import vector2sentence
import numpy as np

Using TensorFlow backend.


In [2]:

model = keras.models.load_model(filepath='./models/lstm_attention_iteration8_units_256_val-acc_0.314.h5')

C:\Users\asus\Anaconda3\lib\site-packages\keras\engine\topology.py:1253: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [3]:
preprocessor = load.Preprocessor(r'./Data')
preprocessor.load_word_indice('word_count_10.tsv')
MAX_SEQUENCE_LENGTH=32

In [4]:
def sample(preds, temperature=0.5):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### Generation without seed.

In [6]:
%%time
for score in [1,2,3,4,5]:
    with open('./Generation/Generated_without_seed_star_%d.txt'%score,encoding='utf-8',mode='w') as f:
    
        for _ in range(200):

            sentence_vector = ['padding']*(MAX_SEQUENCE_LENGTH-1)+['<sor>']
            sentence_vector = sentence2vector(sentence_vector,preprocessor)

            score_one_hot=np.zeros([1,5])
            score_one_hot[0,score-1]=1
            b=score_one_hot
            generated_vector=[]
            while (True):

                a=np.array([sentence_vector])
            #     print("a.shape",a.shape)
                softmax_vector=model.predict([a,b])[0]

                next_index = sample(softmax_vector, temperature=0.7)
            #     print(next_index)
                if next_index == 3:
                    break
                sentence_vector.pop(0)
                sentence_vector.append(next_index)
                generated_vector.append(next_index)

            # print(sentence_vector)
            generated=' '.join(vector2sentence(generated_vector,preprocessor))
            f.write(generated)
            f.write('\n')

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: RuntimeWarning: divide by zero encountered in log


Wall time: 3min 32s


### Generation with random seed